In [ ]:
import sqlite3 

In [ ]:
conn = sqlite3.connect("test_database_")


In [ ]:
cursor = conn.cursor()

In [ ]:
conn.execute("""
  CREATE TABLE estados (
    cod INTEGER PRIMARY KEY,
    nome VARCHAR(50),
    sigla VARCHAR(5)
  )
""")

In [ ]:
import pandas as pd
from typing import Dict, Any
def convert_csv_into_table(
    connection: sqlite3.Connection,
    cursor: sqlite3.Connection.cursor,
    csv_filename: str,
    table_name: str,
    columns_dict: Dict[str, str],
    read_csv_args: Dict[str, Any] = {}
):
  dataframe = pd.read_csv(csv_filename, **read_csv_args)

  insert_execute_string = f'INSERT INTO {table_name} ({{}}) VALUES ({{}});'

  for record in dataframe.to_dict('records'):
    record = {columns_dict[key]: value for key, value in record.items()}
    values_string = ','.join([
      str(i) if type(i) != str else f'"{i}"' for i in record.values()
    ])
    execute_string = insert_execute_string.format(
        ','.join(list(record.keys())),
        values_string
    )
    
    cursor.execute(execute_string)
  conn.commit()


In [ ]:
cursor.execute(
    """
    CREATE TABLE pessoas_habilitadas_nao_processado (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      data VARCHAR(8),
      contagem_pessoas INTEGER
    );
    """
)

In [ ]:
convert_csv_into_table(
    connection=conn,
    cursor=cursor,
    csv_filename='../data/external/pessoas_habilitadas.csv',
    table_name='pessoas_habilitadas_nao_processado',
    columns_dict={
        'Data': 'data',
        'Pessoas de 14 anos ou mais de idade - Pessoa - Instituto Brasileiro de Geografia e Estatística': 'contagem_pessoas'
    })

In [ ]:
cursor.execute(
    "CREATE TABLE pessoas_habilitadas AS "\
    "SELECT * from pessoas_habilitadas_nao_processado WHERE data > \"2019\";"
)

In [ ]:
conn.commit()

In [ ]:
cursor.execute(
    """
    CREATE TABLE pessoas_ocupadas_nao_processado (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      data VARCHAR(8),
      contagem_pessoas INTEGER
    );
    """
)

In [ ]:
convert_csv_into_table(
    connection=conn,
    cursor=cursor,
    csv_filename='../data/external/pessoas_ocupadas.csv',
    table_name='pessoas_ocupadas_nao_processado',
    columns_dict={
        'Data': 'data',
        'Pessoas ocupadas - Pessoa - Instituto Brasileiro de Geografia e Estatística': 'contagem_pessoas'
    })

In [ ]:
cursor.execute(
    "CREATE TABLE pessoas_ocupadas AS "\
    "SELECT * from pessoas_ocupadas_nao_processado WHERE data > \"2019\";"
)

In [ ]:
cursor.execute(
    "CREATE TABLE data_economico AS "\
    "SELECT pessoas_ocupadas.data as data, "\
    "pessoas_ocupadas.contagem_pessoas as contagem_pessoas_ocupadas, "\
    "pessoas_habilitadas.contagem_pessoas as contagem_pessoas_habilitadas "\
    "FROM pessoas_ocupadas INNER JOIN pessoas_habilitadas ON pessoas_ocupadas.data = pessoas_habilitadas.data"
)

In [ ]:
conn.commit()
conn.close()